# Intro
Jeopardy is a popular TV show in the US where participants answer questions to win money. It's been running for many years, and is a major force in popular culture.
Imagine that you want to compete on Jeopardy, and you're looking for any way to win.

The dataset is named jeopardy.csv, and contains 20000 rows from the beginning of a full dataset of Jeopardy questions, which you can download [here](https://www.reddit.com/r/datasets/comments/1uyd0t/200000_jeopardy_questions_in_a_json_file).

In [1]:
import pandas as pd
import re

In [2]:
data = pd.read_csv('jeopardy.csv')
print(data.head(5))

   Show Number    Air Date      Round                         Category  Value  \
0         4680  2004-12-31  Jeopardy!                          HISTORY   $200   
1         4680  2004-12-31  Jeopardy!  ESPN's TOP 10 ALL-TIME ATHLETES   $200   
2         4680  2004-12-31  Jeopardy!      EVERYBODY TALKS ABOUT IT...   $200   
3         4680  2004-12-31  Jeopardy!                 THE COMPANY LINE   $200   
4         4680  2004-12-31  Jeopardy!              EPITAPHS & TRIBUTES   $200   

                                            Question      Answer  
0  For the last 8 years of his life, Galileo was ...  Copernicus  
1  No. 2: 1912 Olympian; football star at Carlisl...  Jim Thorpe  
2  The city of Yuma in this state has a record av...     Arizona  
3  In 1963, live on "The Art Linkletter Show", th...  McDonald's  
4  Signer of the Dec. of Indep., framer of the Co...  John Adams  


In [3]:
print(data.columns)

Index(['Show Number', ' Air Date', ' Round', ' Category', ' Value',
       ' Question', ' Answer'],
      dtype='object')


### Data Cleaning

In [4]:
data.columns = ['Show Number', 'Air Date', 'Round', 'Category',
                'Value', 'Question', 'Answer']

In [5]:
def normalize_text(text):
    text = text.lower()
    text = re.sub("[^A-Za-z0-9\s]", "", text)
    text = re.sub("\s+", " ", text)
    return text

def normalize_values(value):
    value=re.sub('[^A-Za-z0-9\s]','',value)
    try:
        value=int(value)
    except Exception:
        value=0
    return value

In [6]:
data['clean_question'] = data['Question'].apply(normalize_text)
data['clean_answer'] = data['Answer'].apply(normalize_text)
data['clean_value'] = data['Value'].apply(normalize_values)

In [7]:
data['Air Date'] = pd.to_datetime(data['Air Date'])

In [8]:
def count_matches(row):
    split_answer = row["clean_answer"].split()
    split_question = row["clean_question"].split()
    if "the" in split_answer:
        split_answer.remove("the")
    if len(split_answer) == 0:
        return 0
    match_count = 0
    for item in split_answer:
        if item in split_question:
            match_count += 1
    return match_count / len(split_answer)

data["answer_in_question"] = data.apply(count_matches, axis=1)
data["answer_in_question"].mean()

0.059001965249777744

In [9]:
question_overlap = []
terms_used = set()

data = data.sort_values("Air Date")

for i, row in data.iterrows():
        split_question = row["clean_question"].split(" ")
        split_question = [q for q in split_question if len(q) > 5]
        match_count = 0
        for word in split_question:
            if word in terms_used:
                match_count += 1
        for word in split_question:
            terms_used.add(word)
        if len(split_question) > 0:
            match_count /= len(split_question)
        question_overlap.append(match_count)
data["question_overlap"] = question_overlap

data["question_overlap"].mean()

0.6876260592169776

In [10]:
def determine_value(row):
    value = 0
    if row['clean_value'] > 800:
        value = 1
    return value

In [11]:
data['high_value'] = data.apply(determine_value, axis=1)

In [12]:
def count_usage(term):
    low_count = 0
    high_count = 0
    for i, row in data.iterrows():
        if term in row["clean_question"].split(" "):
            if row["high_value"] == 1:
                high_count += 1
            else:
                low_count += 1
    return high_count, low_count

In [13]:
from random import choice

terms_used_list = list(terms_used)
comparison_terms = [choice(terms_used_list) for _ in range(10)]

observed_expected = []

for term in comparison_terms:
    observed_expected.append(count_usage(term))

observed_expected

[(0, 1),
 (0, 1),
 (0, 1),
 (1, 0),
 (1, 0),
 (5, 11),
 (0, 1),
 (2, 0),
 (0, 1),
 (0, 1)]

In [14]:
from scipy.stats import chisquare
import numpy as np

high_value_count = data[data["high_value"] == 1].shape[0]
low_value_count = data[data["high_value"] == 0].shape[0]

chi_squared = []
for obs in observed_expected:
    total = sum(obs)
    total_prop = total / data.shape[0]
    high_value_exp = total_prop * high_value_count
    low_value_exp = total_prop * low_value_count
    
    observed = np.array([obs[0], obs[1]])
    expected = np.array([high_value_exp, low_value_exp])
    chi_squared.append(chisquare(observed, expected))

print(chi_squared)

[Power_divergenceResult(statistic=0.401962846126884, pvalue=0.5260772985705469), Power_divergenceResult(statistic=0.401962846126884, pvalue=0.5260772985705469), Power_divergenceResult(statistic=0.401962846126884, pvalue=0.5260772985705469), Power_divergenceResult(statistic=2.487792117195675, pvalue=0.11473257634454047), Power_divergenceResult(statistic=2.487792117195675, pvalue=0.11473257634454047), Power_divergenceResult(statistic=0.05201920695280218, pvalue=0.8195862285167737), Power_divergenceResult(statistic=0.401962846126884, pvalue=0.5260772985705469), Power_divergenceResult(statistic=4.97558423439135, pvalue=0.025707519787911092), Power_divergenceResult(statistic=0.401962846126884, pvalue=0.5260772985705469), Power_divergenceResult(statistic=0.401962846126884, pvalue=0.5260772985705469)]


# Chi-Squared Results

None of the terms had a significant difference in usage between high value and low value rows. Additionally, the frequencies were all lower than 5, so the chi-squared test isn't as valid. It would be better to run this test with only terms that have higher frequencies.